# Выбор локации для нефтяной скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

______

###### Данное исследование разделено на несколько частей.

* [1. Загрузка и подготовка данных.](#section1)
* [2. Обучение и проверка модели.](#section2)
* [3. Подготовка к расчету пибыли.](#section3)
* [4. Расчет прибыли и рисков.](#section4)
* [5. Выводы.](#section5)

<a id='section1'> </a>

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from numpy.random import RandomState

Даны три файла с данными о геологоразведке трёх регионов. В файлах:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

In [2]:
df0 = pd.read_csv('./datasets/geo_data_0.csv')
df1 = pd.read_csv('./datasets/geo_data_1.csv')
df2 = pd.read_csv('./datasets/geo_data_2.csv')

In [3]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# создадим словаь в котором в качестве ключа будем исользовать название месторождения, а в качестве значения соотв датафрейм
# наличие словаря позволит избежать дублирования одинакового кода для разных датафреймов
df_dict = {'Месторождение 0' : df0, 'Месторождение 1' : df1, 'Месторождение 2' : df2}

In [5]:
for key in df_dict:
    print(key)
    df_dict[key].info()
    print()

Месторождение 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Месторождение 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Месторождение 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data 

Все ячейки в данных заполнены, пропусков нет.

In [6]:
for key in df_dict:
    print(key)
    print(df_dict[key].describe())
    print()
    print()

Месторождение 0
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347


Месторождение 1
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       

Посмотрим на корреляцию параметров f0, f1, f2 с величиной запасов

In [7]:
for key in df_dict:
    print(key)
    print(df_dict[key].corr())
    print()
    print()

Месторождение 0
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


Месторождение 1
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


Месторождение 2
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000




Как видно из таблиц выше наибольшую корреляцию с величиной запасов имеет параметр f2, а на "Месторождении 1" корреляция вообще составляет почти 100%. Следует предположить что в виду высокой корреляции параметров на "Месторождении 1" модель будет предсказывать запасы с более высокой точностью, чем на других месторождениях. 

Разделим имеющиеся датафреймы на тренировочную и валидационную выборки. Аналогично самим датафреймам, тренировочную и валидационную выборку для разных месторождений объединим в словари.  

In [8]:
train_dict = {}
valid_dict = {}

for key in df_dict:
    train_dict[key], valid_dict[key] = train_test_split(df_dict[key], test_size=0.25, random_state=42)
    
    # проверим что суммарные размеры выборок соответствуют исходным датафреймам
    print((len(train_dict[key]) + len(valid_dict[key])) == len(df_dict[key]))

True
True
True


<a id='section2'> </a>

# 2. Обучение и проверка модели

По условиям задачи допускается использовать только модель линейной регрессии. Обучим модель на датафреймах и оценим средний предсказаный запас и СКО оценки запаса

In [9]:
y_col = ['product']
x_col = ['f0', 'f1', 'f2']

model = LinearRegression()
model_rmse_score = {}

for key in train_dict: 
    model.fit(train_dict[key][x_col], train_dict[key][y_col])
    valid_dict[key] = valid_dict[key].copy()
    valid_dict[key]['product_predict'] = model.predict(valid_dict[key][x_col])
    model_rmse_score[key] = np.sqrt(mean_squared_error(valid_dict[key][y_col], valid_dict[key]['product_predict']))
    print(key)
    print('Cредний предсказанный запас сырья: {:.4f} тыс. баррелей'.format(valid_dict[key]['product_predict'].mean()))
    print('СКО оценки запаса сырья по модели: {:.4f} тыс. баррелей'.format(model_rmse_score[key]))
    print()

Месторождение 0
Cредний предсказанный запас сырья: 92.3988 тыс. баррелей
СКО оценки запаса сырья по модели: 37.7566 тыс. баррелей

Месторождение 1
Cредний предсказанный запас сырья: 68.7129 тыс. баррелей
СКО оценки запаса сырья по модели: 0.8903 тыс. баррелей

Месторождение 2
Cредний предсказанный запас сырья: 94.7710 тыс. баррелей
СКО оценки запаса сырья по модели: 40.1459 тыс. баррелей



Как и ожидалось, СКО оценки запасов на "Месторождении 1" ниже чем на других месторождениях. На этом месторождении модель отработала точнее.

<a id='section3'> </a>

# 3. Подготовка к расчёту прибыли

Создадим следующие константы соответствующие условиям задачи:
* RESEARCH_POINTS   - количество исследуемых точек в регионе
* SELECTED_POINTS   - количество точек которые выбирают для разработки
* EARNINGS_PER_THOUSEND_BARRELS   - доход с тысячи баррелей (руб)
* BUDGET   - бюджет на разработку скважин в регионе (руб)

In [10]:
RESEARCH_POINTS = 500
SELECTED_POINTS = 200
EARNINGS_PER_THOUSEND_BARRELS = 450e3
BUDGET = 10e9

Оценим средний достаточный объём сырья в одной скважине для безубыточной разработки

In [11]:
BUDGET / EARNINGS_PER_THOUSEND_BARRELS / SELECTED_POINTS

111.11111111111111

Для безубыточности проекта необходимо чтобы каждая из пробуренных скважен в среднем приносила не менее 111,11 тыс. баррелей. Что выше средних предсказанных моделью значений для всех трех месторождений. Оценим реальные средние запасы на месторождениях

In [12]:
for key in df_dict:
    print(key)
    print('Cредний запас сырья {:.4f} тыс. баррелей'.format(df_dict[key]['product'].mean()))
    print()

Месторождение 0
Cредний запас сырья 92.5000 тыс. баррелей

Месторождение 1
Cредний запас сырья 68.8250 тыс. баррелей

Месторождение 2
Cредний запас сырья 95.0000 тыс. баррелей



Реальные средние оценки запасов близки средним запасам предсказанным моделью и ниже требуемого для безубыточности проекта. Оценим средний запас лучших точек на месторождениях:

In [13]:
for key in df_dict:
    print(key)
    print(df_dict[key].sort_values('product', ascending=False)[:SELECTED_POINTS]['product'].mean(), 'тыс. бареллей')
    print()

Месторождение 0
184.83373964536023 тыс. бареллей

Месторождение 1
137.94540774090612 тыс. бареллей

Месторождение 2
189.5514769817665 тыс. бареллей



Если бы была возможность для каждого месторождения выбрать 200 лучших точек, то все три месторождения были бы окупаемы. Но так как точки выбираются случайно, создадим статистическую модель, которая оценит возможную прибыль и риски для каждого месторождения.

<a id='section4'> </a>

# 4. Расчёт прибыли и рисков 

Создадим функцию income_calc которая принимает на вход датафрейм с реальными и предсказанными запасами на месторождении, случайным образом выбирает 500 точек из них выбирает 200 лучших точек по предсказанию модели и оценивает прибыль. За прибыль принята разность между суммой полученной от продажи всех запасов нефти на выбранных точках и бюджетом на разработку.

In [14]:
state = RandomState(42) 
def income_calc(df):
    selected_index = (df.sample(RESEARCH_POINTS, replace=True, random_state=state)
                        .sort_values('product_predict', ascending=False)[:SELECTED_POINTS]['product_predict']
                        .index)
    total_earnings = df.loc[selected_index]['product'].sum() * EARNINGS_PER_THOUSEND_BARRELS
    income = total_earnings - BUDGET
    return income

Используя созданную функцию рассчитаем прибыль для одной конкретной реализации из 200 точек на каждом месторождении.

In [15]:
for key in df_dict:
    print(key)
    print('{:.2f} млн. руб.'.format(income_calc(valid_dict[key]) / 1e6))
    print()

Месторождение 0
571.78 млн. руб.

Месторождение 1
517.68 млн. руб.

Месторождение 2
327.92 млн. руб.



Для данной реализации все три месторождения оказались прибыльными. Оценим среднюю прибыль и риски при разработки месторождений. Под риском будем понимать возможность получения отрицательной прибыли. Для проведения данного эксперемента оценим возможную прибыль по 1000 реализаций. 

In [16]:
BOOSTRAP_NUM = 1000
income_dict = {}
loss_ctr_dict = {}

for key in valid_dict:
    income_dict[key] = []
    loss_ctr_dict[key] = 0
    for i in range(BOOSTRAP_NUM):
        income = income_calc(valid_dict[key])
        income_dict[key].append(income)
        if income < 0:
            loss_ctr_dict[key] += 1

In [17]:
loss_risk = {}
for key in valid_dict:
    loss_risk[key] = loss_ctr_dict[key] / BOOSTRAP_NUM
    print(key)
    print('Вероятность убытков на месторождении: {:.2%}'.format(loss_risk[key]))
    print()

Месторождение 0
Вероятность убытков на месторождении: 5.90%

Месторождение 1
Вероятность убытков на месторождении: 0.90%

Месторождение 2
Вероятность убытков на месторождении: 7.60%



In [18]:
for key in valid_dict:
    print(key)
    print('Средняя прибыль на месторождении: {:.2f} млн. руб.'.format(pd.Series(income_dict[key]).mean() / 1e6))
    print()

Месторождение 0
Средняя прибыль на месторождении: 399.62 млн. руб.

Месторождение 1
Средняя прибыль на месторождении: 452.57 млн. руб.

Месторождение 2
Средняя прибыль на месторождении: 378.48 млн. руб.



In [19]:
for key in valid_dict:
    print(key)
    print('В 95% случаев доход будет не менее: {:.2f} млн. руб.'.format(pd.Series(income_dict[key]).quantile(0.05) / 1e6))
    print()

Месторождение 0
В 95% случаев доход будет не менее: -14.27 млн. руб.

Месторождение 1
В 95% случаев доход будет не менее: 110.04 млн. руб.

Месторождение 2
В 95% случаев доход будет не менее: -68.07 млн. руб.



Риск убытков менее 2,5% только на "Месторождении 1". Кроме того только на этом месторождении прибыль положительна более чем в 95% случаев. В связи с этим из трех месторождений представленных на анализ для разработки рекомендуется "Месторождение 1".

<a id='section5'> </a>

# Выводы

1. Высокую корреяцию с величиной запасов показывает параметр f2, причем на "Месторождении 1" корреляция почти 100%.
2. При оценки запасов для месторождений 0 и 2 модель линейной регресии имеет высокое значение СКО (37,75 и 40,15 мил. баррелей), что сказывается на точности предсказаний. По условиям задачи допускалось использование только модели линейной регресии, однако использование дополнительнительных моделей могло бы снизить СКО оценок и как следствие помогло бы получить более высокую прибыль (в смысле 95% интервала). В связи с этим в перспективе рекомендуется проработать возможность использования более сложных моделей.
3. Для заданной модели риск убытков менее 2,5% только для "Месторождения 1" (0,9%). С вероятностью 95% прибыль с данного месторождения будет выше 110 млн. руб.
4. Для разработки рекомендуется "Месторождение 1".